# Langfuse Data Analysis

In [110]:
import json
import os
from typing import Optional

from dotenv import load_dotenv
from langfuse import Langfuse

In [111]:
def init_langfuse_client(public_key: str, secret_key: str, host: str):
    return Langfuse(
        public_key=public_key,
        secret_key=secret_key,
        host=host,
    )

In [112]:
def spanning_seconds(end_time, start_time):
    return (end_time - start_time).total_seconds()

In [118]:
def get_all_traces(client, name: Optional[str]=None):
    traces = []
    page = 1

    while True:
        data = client.fetch_traces(name=name, page=page).data
        if len(data) == 0:
            break
        traces += data
        page += 1

    return traces

In [119]:
def pprint_json(data):
    print(json.dumps(json.loads(data), indent=2, ensure_ascii=False))

In [120]:
load_dotenv(".env", override=True)

client = init_langfuse_client(
    os.getenv("LANGFUSE_PUBLIC_KEY"),
    os.getenv("LANGFUSE_SECRET_KEY"),
    os.getenv("LANGFUSE_HOST")
)

In [121]:
traces = get_all_traces(client)

In [122]:
len(traces)

672

In [123]:
results = []
for trace in traces:
    if trace.output.get('metadata', {}).get('error_type', ''):
        results.append(trace)

In [124]:
len(results)

234

In [125]:
error_results = {}
for result in results:
    error_type = result.output['metadata']['error_type']
    if error_type not in error_results:
        error_results[error_type] = [result]
    else:
        error_results[error_type].append(result)

In [126]:
for key, value in error_results.items():
    print(key)
    print(len(value))

SQL_BREAKDOWN_FAILED
91
NO_RELEVANT_SQL
120
OTHERS
1
NO_RELEVANT_DATA
22


OTHERS

In [127]:
for error_result in error_results['OTHERS']:
    pprint_json(error_result.json())

{
  "id": "1e359cf4-3426-4c7d-9013-c5191ec8732d",
  "timestamp": "2024-09-16T15:54:01.832000Z",
  "name": "Ask Question",
  "input": {
    "args": [
      {
        "query": "Which companies in Yfinance have names that starts with C",
        "history": {
          "sql": "SELECT\n  COUNT(DISTINCT \"symbol\") AS \"unique_company_count\"\nFROM\n  \"yfinance_index\" AS \"yfinance_index\"",
          "steps": [
            {
              "sql": "SELECT \"symbol\" AS \"company_symbol\", \"market_cap\" AS \"largest_market_cap\" FROM \"yfinance_report\" AS \"yfinance_report\" WHERE NOT \"market_cap\" IS NULL",
              "summary": "Selects company symbols and their corresponding market capitalizations from the yfinance_report table, filtering out any records where market_cap is NULL.",
              "cte_name": "filtered_yfinance_report"
            },
            {
              "sql": "SELECT \"company_symbol\", \"largest_market_cap\" FROM \"filtered_yfinance_report\" ORDER BY \"large

NO_RELEVANT_DATA

In [128]:
for error_result in error_results['NO_RELEVANT_DATA']:
    pprint_json(error_result.json())

{
  "id": "26221023-c5c1-4282-b4cb-7108d5a21df5",
  "timestamp": "2024-09-16T04:46:19.184000Z",
  "name": "Ask Question",
  "input": {
    "args": [
      {
        "query": "個案年齡大於50的人",
        "history": {
          "sql": "SELECT\n  \"case_info_cms\" AS \"cms_level\"\nFROM\n  \"dbo_cc_case\" AS \"_cc_case\"",
          "steps": [
            {
              "sql": "SELECT \"_cc_case\".\"case_name\" FROM \"dbo_cc_case\" AS \"_cc_case\" JOIN \"dbo_cc_service\" AS \"_cc_service\" ON \"_cc_case\".\"case_id\" = \"_cc_service\".\"case_id\" WHERE LOWER(\"_cc_service\".\"service_signin_time\") < LOWER('08:30:00') AND CURRENT_DATE = CURRENT_DATE",
              "summary": "選取今天簽到時間早於八點半的個案名稱",
              "cte_name": ""
            }
          ],
          "summary": "選取今天簽到時間早於八點半的個案名稱"
        },
        "user_id": null,
        "mdl_hash": "26b43017b12229b8562454e193aa7ec42bd7d9be",
        "thread_id": null,
        "project_id": "80"
      }
    ],
    "kwargs": {}
  },
  "output": {

NO_RELEVANT_SQL

In [129]:
for error_result in error_results['NO_RELEVANT_SQL']:
    pprint_json(error_result.json())

{
  "id": "5080533e-3af2-49c9-a05f-46a2d2914b4a",
  "timestamp": "2024-09-18T08:45:13.836000Z",
  "name": "Ask Question",
  "input": {
    "args": [
      {
        "query": "Count total distinct opens, clicks, and sent emails by date for the Independence Day Email campaign between date 2024-05-01 and 2024-09-30",
        "history": {
          "sql": "SELECT\n  \"date\",\n  COUNT(\"OPEN\") AS \"total_opens\",\n  COUNT(\"CLICK\") AS \"total_clicks\",\n  COUNT(\"SENT\") AS \"total_sent\"\nFROM\n  \"wren_ai_hubspot_testing\"\nWHERE\n  LOWER(\"campaign_name\") = LOWER('Independence Day Email')\nGROUP BY\n  \"date\"\nORDER BY\n  \"date\"",
          "steps": [
            {
              "sql": "SELECT \"date\", \"OPEN\", \"CLICK\", \"SENT\" FROM \"wren_ai_hubspot_testing\" WHERE LOWER(\"campaign_name\") = LOWER('Independence Day Email')",
              "summary": "Selects the date along with the counts of opens, clicks, and sent emails from the wren_ai_hubspot_testing table for the specif